<a href="https://colab.research.google.com/github/szymonszwedzinskiii/DataTemplates/blob/main/LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from sklearn import datasets
import pandas as pd

In [18]:
N, D = 200, 2
SEED = 54321
X,y = datasets.make_classification(n_samples=N,n_features=D,n_informative=2,n_redundant=0,random_state=SEED)

X = np.reshape(X, (N, D))
y = np.reshape(y, (N, 1))



In [19]:
df = pd.DataFrame({
    "X_1": X[:,0],
    "X_2":X[:,1],
    "label": ["positive" if y_i>0 else "negative" for y_i in y]
},index = list(range(N)))

df

,X_1,X_2,label
0,1.434174,-0.593742,negative
1,-0.546980,-1.061760,negative
2,0.651044,0.868971,positive
3,0.249456,-1.033361,negative
4,-1.047607,-0.693791,negative
...,...,...,...
195,-0.922138,-2.105524,negative
196,-1.309470,0.949647,positive
197,0.617753,0.751026,positive
198,-1.028913,-1.058349,negative


In [20]:
import plotly.express as px

In [21]:
px.scatter(df,x="X_1", y="X_2",color="label")

In [44]:
def sigmoid(x):
  return 1/(1 + np.exp(-x))

In [45]:
w= np.array([1.0,2.0])
b = 3.0

w = np.reshape(w, (D,1))


In [46]:
def model(x,w,b):
  logits = np.matmul(x,w) + b
  return sigmoid(logits)

y_pred = model(X,w,b)
print(f" ypred= {list(y_pred)}\n target = {list(y)}")

 ypred= [array([0.96255394]), array([0.58163786]), array([0.99545416]), array([0.7654388]), array([0.63756511]), array([0.90655028]), array([0.68183605]), array([0.99970337]), array([0.99602785]), array([0.99886228]), array([0.99874445]), array([0.98742783]), array([0.99569272]), array([0.12613896]), array([0.99651059]), array([0.99939227]), array([0.98951685]), array([0.9987425]), array([0.36453962]), array([0.1798007]), array([0.48961909]), array([0.99990902]), array([0.99823066]), array([0.99246094]), array([0.08460181]), array([0.9997655]), array([0.93948398]), array([0.97107314]), array([0.19367404]), array([0.76322313]), array([0.8418852]), array([0.99665178]), array([0.98706398]), array([0.91231358]), array([0.93236033]), array([0.57664026]), array([0.07117296]), array([0.9514529]), array([0.55372966]), array([0.99998265]), array([0.98752226]), array([0.95022756]), array([0.94350697]), array([0.99538145]), array([0.98712502]), array([0.99944174]), array([0.28763706]), array([0.0

In [47]:
def plot(w,b):
  import plotly.graph_objects as go
  def get_points(w,b,x1,x2):
    A,B = w[0,0], w[1,0]
    C= b
    return {
        "x":[x1,x2],
        "y":[(C-A*x1)/B, (C-A*x2)/B]
    }
  fig1 = px.scatter(df,x='X_1',y="X_2",color="label")
  fig2= go.Figure(go.Scatter(**get_points(w,b,-2,4), name="brzeg"))
  return go.Figure(data=fig1.data + fig2.data)

In [48]:
plot(w,b)

In [49]:
def cross_entropy(y_pred,target):
  return -np.mean(target * np.log(y_pred) + (1-target) * np.log(1-y_pred))


In [50]:
def compute_costs(X,y,w,b):
  y_pred = model(X,w,b)
  return cross_entropy(y_pred,y)


In [54]:
compute_costs(X,y,w,b)

np.float64(1.027921502033809)

In [60]:
def compute_gradient(X,y,w,b):
  n = len(X)
  y_pred = model(X,w,b)
  dw = (1/n) * np.matmul(X.T,(y_pred-y))
  db = (1/n) * np.sum(y_pred-y)
  return dw,db

In [62]:
compute_gradient(X,y,w,b)



(array([[ 0.20607611],
        [-0.15604127]]),
 np.float64(0.294252928552664))

In [64]:
def step_gradient(data_x, data_y,w_current,b_current,alpha):
  dw,db = compute_gradient(data_x,data_y,w_current,b_current)
  w_new = w_current - alpha * dw
  b_new = b_current - alpha * db
  return w_new,b_new


In [66]:
w_before, b_before = w,b
cost_before = compute_costs(X,y,w_before,b_before)
w_after, b_after = step_gradient(X,y,w_before,b_before,0.1)
cost_after = compute_costs(X,y,w_after,b_after)
print(f"cost before = {cost_before}\n cost after = {cost_after}")

cost before = 1.027921502033809
 cost after = 1.0126587031707026


In [67]:
w_before, b_before = w,b
cost_before = compute_costs(X,y,w_before,b_before)
w_after, b_after = step_gradient(X,y,w_before,b_before,1.0)
cost_after = compute_costs(X,y,w_after,b_after)
print(f"cost before = {cost_before}\n cost after = {cost_after}")

cost before = 1.027921502033809
 cost after = 0.8823548482348891


In [68]:
print(w_before,b_before)
print(w_after,b_after)

[[1.]
 [2.]] 3.0
[[0.79392389]
 [2.15604127]] 2.705747071447336


Plot before 1 step

---



In [71]:
plot(w_after,b_after)

Plot after 1 step

In [70]:
plot(w_before,b_before)

In [75]:
def training(data_x,data_y,w_init,b_init,alpha,num_eopchs):
  w=w_init
  b = b_init

  cost_history=[]

  w_history=[]
  b_history =[]
  for i in range(num_eopchs):
    cost_history.append(compute_costs(data_x,data_y,w,b))
    w_history.append(w)
    b_history.append(b)
    w,b = step_gradient(data_x,data_y,w,b,alpha)
    cost = compute_costs(data_x,data_y,w,b)
    cost_history.append(cost)
  return cost_history, w_history,b_history


In [76]:
cost_history, w_history,b_history = training(X,y,w,b,0.1,20)
print(f'first cost = {cost_history[0]}, cost after 20 epochs and alpha =0.01 = {cost_history[-1]}')

first cost = 1.027921502033809, cost after 20 epochs = 0.773223216434235


In [77]:
cost_history, w_history,b_history = training(X,y,w,b,0.1,100)
print(f'first cost = {cost_history[0]}, cost after 100 and alpha =0.01 epochs = {cost_history[-1]}')

first cost = 1.027921502033809, cost after 20 epochs = 0.3783349737244731


In [79]:
cost_history, w_history,b_history = training(X,y,w,b,1,20)
print(f'first cost = {cost_history[0]}, cost after 20 epochs and alpha =1 = {cost_history[-1]}')

first cost = 1.027921502033809, cost after 20 epochs and alpha =1 = 0.29649513318300824


Plot for trained data with cost ~0.2964

In [84]:
plot(w_history[-1],b_history[-1])

plot for not trained data

In [85]:
plot(w,b)